In [3]:
import pandas as pd
import numpy as np

# Chapter 7

## 7.1 Handing Missing Data

for numeric data, pandas uses the floating-point value NaN (Not a Number) to represent missing data. we call this a _sentinel value_

In [4]:
string_data = pd.Series(['aardvark', 'artichoke', np.nan, 'avocado'])
string_data

0     aardvark
1    artichoke
2          NaN
3      avocado
dtype: object

In [5]:
string_data.isnull()

0    False
1    False
2     True
3    False
dtype: bool

In [6]:
string_data[0] = None #None value is also trated as NA in object arrays

In [7]:
string_data.isnull()

0     True
1    False
2     True
3    False
dtype: bool

### Filtering Out Missing Data

a few ways to filter out missing data - can always do it by hand using .isnull and boolean indexing. the dropna can be useful. on a Series, it returns the Series with only the non-null data and index values:

In [8]:
from numpy import nan as NA

In [9]:
data = pd.Series([1, NA, 3.5, NA, 7])

In [10]:
data.dropna()

0    1.0
2    3.5
4    7.0
dtype: float64

**_equivalent to_**

In [11]:
data[data.notnull()]

0    1.0
2    3.5
4    7.0
dtype: float64

DataFrames are more complex. dropna by default drops any row containing a missing value:

In [12]:
data = pd.DataFrame([[1., 6.5, 3.], [1., NA, NA], [NA, NA, NA], [NA, 6.5, 3.]])

In [13]:
cleaned = data.dropna()

In [14]:
data

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,6.5,3.0


In [15]:
cleaned

,0,1,2
0,1.0,6.5,3.0


In [16]:
data.dropna(how='all') #passing how='all' will only drop rows that are all NA

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
3,NaN,6.5,3.0


to drop columns in the same way, pass axis=1

In [17]:
data[4] = NA

In [18]:
data

,0,1,2,4
0,1.0,6.5,3.0,NaN
1,1.0,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN
3,NaN,6.5,3.0,NaN


In [19]:
data.dropna(axis=1, how='all')

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,6.5,3.0


suppose you want to keep only rows containing a certain number of observations. you can indicate this with the thresh argument:

In [20]:
df = pd.DataFrame(np.random.randn(7, 3))

In [21]:
df.iloc[:4, 1] = NA

In [23]:
df.iloc[:2, 2] = NA

In [24]:
df

,0,1,2
0,1.530258,NaN,NaN
1,-0.990702,NaN,NaN
2,1.615311,NaN,0.141599
3,-0.070499,NaN,4.344063
4,0.762107,-1.347476,0.403436
5,0.738771,0.796897,-0.511515
6,-0.790316,-0.590541,-0.566759


In [25]:
df.dropna()

,0,1,2
4,0.762107,-1.347476,0.403436
5,0.738771,0.796897,-0.511515
6,-0.790316,-0.590541,-0.566759


In [26]:
df.dropna(thresh=2) #drops rows with more than 2 NAs

,0,1,2
2,1.615311,NaN,0.141599
3,-0.070499,NaN,4.344063
4,0.762107,-1.347476,0.403436
5,0.738771,0.796897,-0.511515
6,-0.790316,-0.590541,-0.566759


### Filling In Missing Data

may want to fill "holes" in data rather than discard. Using fillna can use a constant to replace missing values

In [27]:
df.fillna(0)

,0,1,2
0,1.530258,0.000000,0.000000
1,-0.990702,0.000000,0.000000
2,1.615311,0.000000,0.141599
3,-0.070499,0.000000,4.344063
4,0.762107,-1.347476,0.403436
5,0.738771,0.796897,-0.511515
6,-0.790316,-0.590541,-0.566759


In [28]:
df.fillna({1: 0.5, 2: 0}) #calling fillna with a dict, you can use a different fill value for each column:

,0,1,2
0,1.530258,0.500000,0.000000
1,-0.990702,0.500000,0.000000
2,1.615311,0.500000,0.141599
3,-0.070499,0.500000,4.344063
4,0.762107,-1.347476,0.403436
5,0.738771,0.796897,-0.511515
6,-0.790316,-0.590541,-0.566759


fillna returns a new object, but you can modify the existing object in-place:

In [29]:
_ = df.fillna(0, inplace=True)

In [30]:
df

,0,1,2
0,1.530258,0.000000,0.000000
1,-0.990702,0.000000,0.000000
2,1.615311,0.000000,0.141599
3,-0.070499,0.000000,4.344063
4,0.762107,-1.347476,0.403436
5,0.738771,0.796897,-0.511515
6,-0.790316,-0.590541,-0.566759


same interpolation methods available for reindexing can be used with fillna:


In [31]:
df = pd.DataFrame(np.random.randn(6, 3))

In [33]:
df.iloc[2:, 1] = NA

In [34]:
df.iloc[4:, 2] = NA

In [35]:
df

,0,1,2
0,-0.645449,-0.244689,-2.237666
1,0.067085,0.674679,2.095682
2,0.547922,NaN,0.657636
3,-2.437439,NaN,0.856235
4,-0.925253,NaN,NaN
5,0.518515,NaN,NaN


In [36]:
df.fillna(method='ffill')

,0,1,2
0,-0.645449,-0.244689,-2.237666
1,0.067085,0.674679,2.095682
2,0.547922,0.674679,0.657636
3,-2.437439,0.674679,0.856235
4,-0.925253,0.674679,0.856235
5,0.518515,0.674679,0.856235


In [37]:
df.fillna(method='ffill', limit=2)

,0,1,2
0,-0.645449,-0.244689,-2.237666
1,0.067085,0.674679,2.095682
2,0.547922,0.674679,0.657636
3,-2.437439,0.674679,0.856235
4,-0.925253,NaN,0.856235
5,0.518515,NaN,0.856235


with fillna you can do lots of things like pass the mean or median value of aSeries:

In [38]:
data = pd.Series([1., NA, 3.5, 7])

In [39]:
data.fillna(data.mean())

0    1.000000
1    3.833333
2    3.500000
3    7.000000
dtype: float64

## 7.2 Data Transformation

### Removing Duplicates

In [40]:
data = pd.DataFrame({'k1': ['one', 'two'] * 3 + ['two'], 
                    'k2': [1, 1, 2, 3, 3, 4, 4]})

#example

In [41]:
data # example of DF with dupes

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4
6,two,4


In [42]:
data.duplicated() #duplicated returns a boolean Series indicating whether each row is a duple or not

0    False
1    False
2    False
3    False
4    False
5    False
6     True
dtype: bool

In [43]:
data.drop_duplicates() #drop_duplicates returns a DataFrame where the duplicated array is False:

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4


both methods consider all of the columns by default. you can specify any subset of them to detect duplicates. 

In [44]:
data['v1'] = range(7)

In [45]:
data.drop_duplicates(['k1']) #filter duplicates only based on the 'k1' column

,k1,k2,v1
0,one,1,0
1,two,1,1


_duplicated_ and _drop_duplicates_ by default keep the first observed value combination. passing _keep='last'_ will return the last one

In [49]:
data.drop_duplicates(['k1', 'k2'], keep='last')

,k1,k2,v1
0,one,1,0
1,two,1,1
2,one,2,2
3,two,3,3
4,one,3,4
6,two,4,6


### Transforming Data Using a Function or Mapping

In [50]:
data = pd.DataFrame({'food': ['bacon', 'pulled pork', 'bacon', 
                              'Pastrami', 'corned beef', 'Bacon', 
                              'pastrami', 'honey ham', 'nova lox'],
                    'ounces': [4, 3, 12, 6, 7.5, 8, 3, 5, 6]})

In [51]:
data #example

,food,ounces
0,bacon,4.0
1,pulled pork,3.0
2,bacon,12.0
3,Pastrami,6.0
4,corned beef,7.5
5,Bacon,8.0
6,pastrami,3.0
7,honey ham,5.0
8,nova lox,6.0


_add a column indicating the type of animal that each food came from_   
mapping of each distinct meat type to kind of animal

In [52]:
meat_to_animal = {
    'bacon': 'pig', 
    'pulled pork': 'pig',
    'pastrami': 'cow',
    'corned beef': 'cow', 
    'honey ham': 'pig',
    'nova lox': 'salmon'
}

map method on a series accepts a function or dict-like object containing a mapping, but the problem is some meats are capitalized and other not. so we need to convert each value to lowercase using the str.lower Series method:

In [53]:
lowercased = data['food'].str.lower()

In [54]:
lowercased

0          bacon
1    pulled pork
2          bacon
3       pastrami
4    corned beef
5          bacon
6       pastrami
7      honey ham
8       nova lox
Name: food, dtype: object

In [55]:
data['animal'] = lowercased.map(meat_to_animal)

In [56]:
data

,food,ounces,animal
0,bacon,4.0,pig
1,pulled pork,3.0,pig
2,bacon,12.0,pig
3,Pastrami,6.0,cow
4,corned beef,7.5,cow
5,Bacon,8.0,pig
6,pastrami,3.0,cow
7,honey ham,5.0,pig
8,nova lox,6.0,salmon


using map is a convenient way to perform element-wise transformations and other data cleaning-related operations

### Replacing Values

replace provides simpler and more flexible way to modify a subset of values in an object

In [57]:
data = pd.Series([1., -999., 2., -999., -1000., 3.])

In [58]:
data

0       1.0
1    -999.0
2       2.0
3    -999.0
4   -1000.0
5       3.0
dtype: float64

the -999 values might be sentinel values for missing data. use replace to produce a new Series to replace -999 with NA

In [59]:
data.replace(-999, np.nan)

0       1.0
1       NaN
2       2.0
3       NaN
4   -1000.0
5       3.0
dtype: float64

**replace multiple values at one by passing a list then substitute value:**


In [60]:
data.replace([-999, -1000], np.nan)

0    1.0
1    NaN
2    2.0
3    NaN
4    NaN
5    3.0
dtype: float64

**use a different replacement for each value, pass a list of substitutes**

In [61]:
data.replace([-999, -1000], [np.nan, 0])

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

**the argument passed can also be a dict**

In [62]:
data.replace({-999: np.nan, -1000: 0})

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

**_.replace is distinct from .str.replace_**


### Renaming Axis Indexes

In [68]:
data = pd.DataFrame(np.arange(12).reshape((3, 4)),
                   index=['Ohio', 'Colorado', 'New York'],
                   columns=['one', 'two', 'three', 'four'])

In [69]:
transform = lambda x: x[:4].upper() #like a series the axis indexes have a map method

In [70]:
data.index.map(transform)

Index(['OHIO', 'COLO', 'NEW '], dtype='object')

In [71]:
data.index = data.index.map(transform) #you can assign to index, modifying the DataFrame in place

In [72]:
data


,one,two,three,four
OHIO,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


In [74]:
data.rename(index=str.title, columns=str.upper) 
#create a transformed version of a dataset without modding the original by using rename

,ONE,TWO,THREE,FOUR
Ohio,0,1,2,3
Colo,4,5,6,7
New,8,9,10,11


rename can be used in conjunction with a dict-like object providing new values for a subset of the axis labels:

In [75]:
data.rename(index={'OHIO': 'INDIANA'}, columns={'three': 'peekaboo'})

,one,two,peekaboo,four
INDIANA,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


rename saves you from copying the DF manually and assigning to its index and columns attributes. you can modify a dataset in place pass inplace=True

In [76]:
data.rename(index={'OHIO': 'INDIANA'}, inplace=True)

In [77]:
data

,one,two,three,four
INDIANA,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


### Discretization and Binning